### the following table informs about decision making factors to play tennis at outside for previous 14 days.
classification Problem , can use both CART :(giniimpurity) , ID3 algorithm   
CART is the most efficient algorithm

In [4]:
#CART
import pandas as pd

In [5]:
data = [['Sunny','Hot','High','Weak','No'],
        ['Sunny','Hot','High','Strong','No'],
        ['Overcast','Hot','High','Weak','Yes'],
        ['Rain','Mild','High','Weak','Yes'],
        ['Rain','Cool','Normal','Weak','Yes'],
        ['Rain','Cool','Normal','Strong','No'],
        ['Overcast','Cool','Normal','Strong','Yes'],
        ['Sunny','Mild','High','Weak','No'],
        ['Sunny','Cool','Normal','Weak','Yes'],
        ['Rain','Mild','Normal','Weak','Yes'],
        ['Sunny','Mild','Normal','Strong','Yes'],
        ['Overcast','Mild','High','Strong','Yes'],
        ['Overcast','Hot','Normal','Weak','Yes'],
        ['Rain','Mild','High','Strong','No']]

In [6]:
title = ['Outlook','Temp.','Humidity','Wind','Decision']

In [7]:
Data = pd.DataFrame(data = data,columns = title)
Data

,Outlook,Temp.,Humidity,Wind,Decision
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [8]:
#uniques values in the column
def unique_values(rows,col):
    return set([row[col] for row in rows])

In [9]:
unique_values(data,0)

{'Overcast', 'Rain', 'Sunny'}

In [10]:
#counting the attributes of the features
def count_values(rows):
    count = {}
    for row in rows:
        #print(row)
        label = row[-1]
        if label not in count:
            count[label] = 1
        else:
            count[label] += 1
    return count

In [11]:
count_values(data)

{'No': 5, 'Yes': 9}

In [12]:
#forming Question : class
class Question:
    def __init__(self,column,value):
        self.column = column
        self.value = value
    
    def match(self,example):
        val = example[self.column]
        return val == self.value
    
    def __repr__(self):
        condition = '=='
        return ('Is %s %s %s ?'%(title[self.column],condition,self.value))

In [13]:
q = Question(1,'Hot')
q

Is Temp. == Hot ?

In [14]:
r = Question(0,'Sunny')
r

Is Outlook == Sunny ?

In [15]:
s = Question(3,'Strong')
s

Is Wind == Strong ?

In [16]:
t = Question(2,'Normal')
t

Is Humidity == Normal ?

In [17]:
#Splitting the data with respect to question
def partition(rows,question):
    true_rows,false_rows = [],[]
    for row in rows:
        #print(row)
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows

In [18]:
t_r , f_r = partition(data,Question(0,'Sunny'))

In [19]:
print('true branch :',t_r)
print('false branch:',f_r)

true branch : [['Sunny', 'Hot', 'High', 'Weak', 'No'], ['Sunny', 'Hot', 'High', 'Strong', 'No'], ['Sunny', 'Mild', 'High', 'Weak', 'No'], ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'], ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes']]
false branch: [['Overcast', 'Hot', 'High', 'Weak', 'Yes'], ['Rain', 'Mild', 'High', 'Weak', 'Yes'], ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'], ['Rain', 'Cool', 'Normal', 'Strong', 'No'], ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'], ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'], ['Overcast', 'Mild', 'High', 'Strong', 'Yes'], ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'], ['Rain', 'Mild', 'High', 'Strong', 'No']]


In [20]:
#Visualizing whether this true branch is right or wrong
d = pd.DataFrame(t_r,columns = title)
print(d)
print()
c = pd.DataFrame(f_r,columns = title)
print(c)

  Outlook Temp. Humidity    Wind Decision
0   Sunny   Hot     High    Weak       No
1   Sunny   Hot     High  Strong       No
2   Sunny  Mild     High    Weak       No
3   Sunny  Cool   Normal    Weak      Yes
4   Sunny  Mild   Normal  Strong      Yes

    Outlook Temp. Humidity    Wind Decision
0  Overcast   Hot     High    Weak      Yes
1      Rain  Mild     High    Weak      Yes
2      Rain  Cool   Normal    Weak      Yes
3      Rain  Cool   Normal  Strong       No
4  Overcast  Cool   Normal  Strong      Yes
5      Rain  Mild   Normal    Weak      Yes
6  Overcast  Mild     High  Strong      Yes
7  Overcast   Hot   Normal    Weak      Yes
8      Rain  Mild     High  Strong       No


In [21]:
#quantifying methods to get a best question and to best splitting the data
#foe splitting we use gini index or entropy
#here, Gini Index (for class label : Decision)
def gini(rows):
    count = count_values(rows)
    gini = 1
    for val in count:
        p = count[val]/len(rows)
        gini -= p**2
    return gini        

#or entropy
def entropy(rows):
    entropy = 0
    count = count_values(rows)
    #print(count)
    from math import log
    log2 = lambda x : log(x)/log(2)
    for val in count:
        p = count[val]/len(rows)
        entropy -= p*log2(p)
    return entropy

In [22]:
gini(data)

0.4591836734693877

In [23]:
entropy(t_r)        

0.9709505944546686

In [24]:
#to find the best question for splitting we use Information Gain ,can use both gini ans well as entropy
def info_gain_entropy(current,left,right):
    p = float(len(left))/(len(left)+len(right))
    return current - p * entropy(left) - (1-p) * entropy(right)
#or gini 


def info_gain_gini(current,left,right):
    p = float(len(left))/(len(left)+len(right))
    return current - (p*gini(left) + (1-p)*gini(right))


In [44]:
#Information gain for wind
current_uncertaininty=entropy(data)
info_gain_entropy(current_uncertaininty,t_r,f_r)    

0.1022435636098506

In [45]:
# finding the best split  (best gain and best question)

#RootNode
def find_best_split(rows):
    best_gain=0
    best_question=None
    #calculate the current_gain
    current=entropy(rows)
    #total number of features
    features=len(rows[0])-1
    for col in range(features):
        #collects all unique classes for a feature
        values=set([row[col] for row in rows])
        #print(values)
        for val in values:
            #traverse each unique classs
            #ask the corresponding question
            question=Question(col,val)
            #devide the data based on that ques
            true_rows,false_rows=partition(rows,question)
            if len(true_rows)==0 or len(false_rows) ==0:
                #no use go to next iteration
                continue
            #calculate corresponding gain
            gain=info_gain_entropy(current,true_rows,false_rows)
            #if gain is > than the best replace
            if gain>=best_gain:
                best_gain,best_question=gain,question
            #iterate through each unique class of each feature and return the best gain and best question     
    return best_gain,best_question


In [46]:
len(data[0])

5

In [47]:
g,q = find_best_split(data)
print('Root Node .....')
print('Question:',q)
print('gain:',g)

Root Node .....
Question: Is Outlook == Overcast ?
gain: 0.22600024438491662


In [48]:
#buliding the tree
class DecisionTree:
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
                

In [49]:
class Leaf:
    def __init__(self,rows):
        self.predictions = count_values(rows)

In [50]:
def build_tree(rows):
    #getting best gain and question from the previous function
    
    
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain , question = find_best_split(rows)
    
    if gain == 0:
        return Leaf(rows)
    #splitting the data for the question given
    true_rows , false_rows = partition(rows,question)
    
    #finding the leaf node recursively
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    #returning question node
    return DecisionTree(question,true_branch,false_branch)

In [51]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [52]:
my_tree = build_tree(data)

In [53]:
print_tree(my_tree)

Is Outlook == Overcast ?
--> True:
  Predict {'Yes': 4}
--> False:
  Is Humidity == High ?
  --> True:
    Is Outlook == Rain ?
    --> True:
      Is Wind == Strong ?
      --> True:
        Predict {'No': 1}
      --> False:
        Predict {'Yes': 1}
    --> False:
      Predict {'No': 3}
  --> False:
    Is Wind == Strong ?
    --> True:
      Is Temp. == Mild ?
      --> True:
        Predict {'Yes': 1}
      --> False:
        Predict {'No': 1}
    --> False:
      Predict {'Yes': 3}


In [35]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [36]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [37]:

#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(data[0], my_tree)

{'No': 3}

In [42]:
testing_data = [['Sunny','Hot','Normal','Strong','Yes'],
        ['Rain','Mild','High','Strong','No'],
        ['Rain','Mild','Normal','Strong','Yes'],
        ['Overcast','Mild','High','Strong','No'],
        ['Overcast','Hot','Normal','Weak','No'],
        ['Overcast','Mild','Normal','Weak','Yes']]

In [43]:
print(title)
print()
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

['Outlook', 'Temp.', 'Humidity', 'Wind', 'Decision']

Actual: Yes. Predicted: {'No': '100%'}
Actual: No. Predicted: {'No': '100%'}
Actual: Yes. Predicted: {'Yes': '100%'}
Actual: No. Predicted: {'Yes': '100%'}
Actual: No. Predicted: {'Yes': '100%'}
Actual: Yes. Predicted: {'Yes': '100%'}
